## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
import os
from xlrd import *
import csv 
import sys
import win32com.client as w3c
from tempfile import NamedTemporaryFile

In [2]:
PATH = 'C:/Users/smsoh/Documents/GitHub/CLEF2021/task3/data/Task3a_Batch_1.xlsx'
PASS = 'fake@2021'

In [3]:
xlapp = w3c.Dispatch("Excel.Application")
xlwb = xlapp.Workbooks.Open(PATH, False, True, None, PASS)

In [4]:
#df = pd.read_csv("data/Task3a_Batch_1.xlsx")
f = NamedTemporaryFile(delete=False, suffix='.csv')
f.close()
os.unlink(f.name)

xlCSVWindows = 0x17
xlwb.SaveAs(Filename=f.name, FileFormat=xlCSVWindows)
df = pd.read_csv(f.name)
df

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false
...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false


In [5]:
df.head()

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false


In [8]:
df['label'] = df['our rating'].apply(lambda x: 0 if 'true' in x.lower() else ( 1 if 'partially false' in x.lower() else ( 2 if 'false' in x.lower() else 3)))
df

,public_id,text,title,our rating,label
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,2
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false,1
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false,1
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false,2
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false,2
...,...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false,2
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false,2
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false,2
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false,2


In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
X.shape

(437, 4)

In [13]:
y.shape

(437,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.4.1'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'Missouri lawmakers condemn Las Vegas shooting'

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smsoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
corpus

rn hate crime',
 'trump blame forget wear mask contract covid',
 'break disneyland abandon california move texa',
 'fox news win pulitz prize report crisi daili worl',
 'one five dad seen kid father day survey find',
 'shock reason pfizer coronaviru vaccin requir storag c contain experiment nanotech compon never use vaccin',
 'joke ilhan omar tell resid regist vote minnesota',
 'alabama suprem court recogn unborn person landmark rule',
 'henri kissing say solut coronaviru new world order',
 'promis usc research discov regener entir immun system hour',
 'itali arrest cardin seiz vatican citi',
 'obama legal impeach presid appoint judg',
 'offici site governor toni ever',
 'dr jill biden american requir learn spanish win',
 'armi scientist success teleport soldier massachusett germani',
 'america biggest owner farmland bill gate',
 'catastroph effect rail privatis reveal year',
 'border polit san diego democrat peter say trump stop beat mexico',
 'hous democrat move remov nation anthem s

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

259, 1428, 1694],
 [1745, 2455, 1694, 1194, 2712, 2992, 4020, 1706, 3237],
 [4988, 4835, 4627, 409, 738],
 [3236, 887, 809, 4936, 4674, 2550, 3806, 2038, 1341, 2531, 3996],
 [3631, 2883, 4066, 4627, 1694, 4973, 3657, 643],
 [1139, 96, 2050, 103, 4073, 3950, 1565, 1755, 2633, 1892, 617],
 [4359, 972, 3588, 4547, 3611, 3831, 75, 2420, 4020],
 [2550, 4948, 421, 2365, 4948, 77, 742, 777, 2742, 3495, 1668, 1436],
 [4351, 2227, 4095, 2130, 4917, 4371, 2699],
 [3561, 1465, 4225, 194, 1489, 2088, 509, 1132, 4077],
 [1500, 4840, 3602, 1436],
 [1117, 2478, 1530, 19, 413, 3397, 3099],
 [3777, 3629, 4157, 1348, 1846, 1259, 2105, 4435],
 [176, 108, 3969, 1760, 4312, 1754, 2631, 388, 3914, 720, 146, 43],
 [1546, 4644, 3527, 2550, 4193, 1843, 2038, 3432, 2420],
 [823, 3698, 1227, 4402, 2200, 3352, 286, 312, 3561, 1493],
 [3495, 1644, 1096, 3800, 4371, 885],
 [2807,
  4835,
  918,
  2161,
  2831,
  4836,
  3709,
  2965,
  3528,
  4310,
  4654,
  3528,
  4349,
  4864],
 [4838, 919, 1332, 1249, 4619, 13

### Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3214 4873  286]
 [   0    0    0 ... 1772 2793 3679]
 [   0    0    0 ... 1563 2663 1521]
 ...
 [   0    0    0 ... 3568 4478 3528]
 [   0    0    0 ... 1080 2362 4269]
 [   0    0    0 ... 3611 4171 3871]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 4477,  521, 3048, 4616, 2566, 3214, 4873,  286])

In [28]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape

(437, (437,))

In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [31]:
X_final.shape,y_final.shape

((437, 20), (437,))

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [33]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
5/5 [==============================] - 6s 779ms/step - loss: 0.6647 - accuracy: 0.1586 - val_loss: 0.5334 - val_accuracy: 0.1586
Epoch 2/50
5/5 [==============================] - 0s 42ms/step - loss: 0.4574 - accuracy: 0.2087 - val_loss: 6.7145e-04 - val_accuracy: 0.1586
Epoch 3/50
5/5 [==============================] - 0s 42ms/step - loss: -0.4027 - accuracy: 0.2007 - val_loss: -2.7095 - val_accuracy: 0.1586
Epoch 4/50
5/5 [==============================] - 0s 39ms/step - loss: -3.5328 - accuracy: 0.1968 - val_loss: -5.4952 - val_accuracy: 0.1586
Epoch 5/50
5/5 [==============================] - 0s 47ms/step - loss: -6.1389 - accuracy: 0.1870 - val_loss: -7.0594 - val_accuracy: 0.1586
Epoch 6/50
5/5 [==============================] - 0s 43ms/step - loss: -6.9430 - accuracy: 0.2513 - val_loss: -8.3575 - val_accuracy: 0.1586
Epoch 7/50
5/5 [==============================] - 0s 42ms/step - loss: -8.7847 - accuracy: 0.1918 - val_loss: -9.4388 - val_accuracy: 0.1586
Epoch 8/50
5

### Adding Dropout 

In [34]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [35]:
y_pred=model.predict_classes(X_test)

C:\Users\smsoh\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test,y_pred)

array([[ 4,  4,  0,  0],
       [11, 12,  0,  0],
       [63, 46,  0,  0],
       [ 3,  2,  0,  0]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.1103448275862069